# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

#to slow down API requests
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_file = "../output_data/cities.csv"
weather_df = pd.read_csv(weather_file)
weather_df.head()

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Kiunga,PG,-6.1219,141.2906,86.86,61,59,2.66,1619831275
1,Ginir,ET,7.1333,40.7000,56.46,87,95,3.89,1619831276
2,Portland,US,45.5234,-122.6762,66.20,59,90,3.44,1619830862
3,Esperance,AU,-33.8667,121.9000,55.40,94,90,6.91,1619831276
4,Vaini,TO,-21.2000,-175.2000,78.80,89,75,12.66,1619831277


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key = g_key)
humidity = weather_df["Humidity"].astype(float)
locations = weather_df[["Lat", "Long"]].astype(float)


In [9]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions
narrowed_lowtemp_df = weather_df.loc[weather_df['Max Temp'] >= 70]
narrowed_hightemp_df = narrowed_lowtemp_df.loc[narrowed_lowtemp_df['Max Temp'] <= 80]
narrowed_wind_df = narrowed_hightemp_df.loc[narrowed_hightemp_df['Wind Speed'] <= 10]
narrowed_cities_df = narrowed_wind_df.loc[narrowed_wind_df['Cloudiness'] == 0] 
narrowed_cities_df.head()

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
6,Atuona,PF,-9.8000,-139.0333,79.41,68,0,3.47,1619831017
126,Canguaretama,BR,-6.3800,-35.1289,77.00,88,0,8.05,1619831319
212,Saint-Philippe,RE,-21.3585,55.7679,73.99,78,0,6.91,1619831348
223,Sambava,MG,-14.2667,50.1667,74.89,88,0,8.03,1619831351
273,Hefei,CN,31.8639,117.2808,70.00,59,0,2.24,1619831359


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.
hotel_df = narrowed_cities_df.loc[:, ["Lat", "Long", "City", "Country"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Lat,Long,City,Country,Hotel Name
6,-9.8000,-139.0333,Atuona,PF,
126,-6.3800,-35.1289,Canguaretama,BR,
212,-21.3585,55.7679,Saint-Philippe,RE,
223,-14.2667,50.1667,Sambava,MG,
273,31.8639,117.2808,Hefei,CN,


In [26]:
# find the closest hotel of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "hotel",
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key,}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get info from hotel_df
    lat = row["Lat"]
    lng = row["Long"]
    city_name = row["City"]

    # add keyword to params dict
    params['location'] = f"{lat}, {lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel name'] = results[0]['name']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")    
    
    time.sleep(1)
        
print("------------End of search------------")

Retrieving Results for Index 6: Atuona.
Closest hotel in Atuona is Hôtel Hiva Oa Hanakee Lodge.
------------
Retrieving Results for Index 126: Canguaretama.
Missing field/result... skipping.
------------
Retrieving Results for Index 212: Saint-Philippe.
Closest hotel in Saint-Philippe is Les Embruns Du Baril.
------------
Retrieving Results for Index 223: Sambava.
Closest hotel in Sambava is Hôtel Las Palmas.
------------
Retrieving Results for Index 273: Hefei.
Closest hotel in Hefei is Hilton Hefei.
------------
Retrieving Results for Index 283: Buraidah.
Closest hotel in Buraidah is Mövenpick Hotel Qassim.
------------
Retrieving Results for Index 289: Kiama.
Closest hotel in Kiama is The Sebel Kiama Harbourside.
------------
Retrieving Results for Index 346: Touros.
Closest hotel in Touros is Hotel Vila Gale Touros.
------------
Retrieving Results for Index 424: Saint-Pierre.
Closest hotel in Saint-Pierre is Villa Delisle, Hotel & Spa.
------------
Retrieving Results for Index 444:

In [29]:
#view updated df
hotel_df.head()

,Lat,Long,City,Country,Hotel Name,Hotel name
6,-9.8000,-139.0333,Atuona,PF,,Hôtel Hiva Oa Hanakee Lodge
126,-6.3800,-35.1289,Canguaretama,BR,,NaN
212,-21.3585,55.7679,Saint-Philippe,RE,,Les Embruns Du Baril
223,-14.2667,50.1667,Sambava,MG,,Hôtel Las Palmas
273,31.8639,117.2808,Hefei,CN,,Hilton Hefei


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [ ]:
# Add marker layer ontop of heat map
# Create a map using state centroid coordinates to set markers
marker_locations = census_data[['Latitude', 'Longitude']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig

# Display figure
